In [ ]:
# csv 파일을 여러 번 읽어 json 파일까지 병합하는 과정

In [ ]:
# 필요한 패키지(라이브러리)를 구글 드라이버에 설치하였는가?
IS_INSTALL_ENV = True
IS_COLAB_ENV = True

In [ ]:

import os, sys  
from google.colab import drive  
drive.mount('/content/drive')   

# colab의 os 폴더 위치
pg_path = '/content/dataset' 

# if IS_NOT_INSTALL_ENV == True:
# Google Drive에서 설치된 파일 위치 - 미리 해당 폴더를 생성해야 한다.
os.symlink('/content/drive/My Drive/Colab Notebooks/Project/AI 속기사/dataset/', pg_path)  
sys.path.insert(0, pg_path)  


# /content/drive/My Drive/Colab Notebooks/Project/AI 속기사/dataset/KlecSpeech/

Mounted at /content/drive


In [ ]:
# colab의 os 폴더 위치
pg_path = '/content/packages' 

# if IS_NOT_INSTALL_ENV == True:
# Google Drive에서 설치된 파일 위치 - 미리 해당 폴더를 생성해야 한다.
os.symlink('/content/drive/My Drive/Colab Notebooks/packages', pg_path)  
sys.path.insert(0, pg_path)  

In [ ]:
!pip3 install unicode
!pip show unicode



if IS_INSTALL_ENV == False:
# !pip install transformers 대신 영구 설치
# os.symlink로 걸어놓은 위치에서 pg_path로 연결된 폴더에 영구 설치한다. 1번만 설치하면 2번 설치할 필요 없음.
    !pip install --target=$pg_path datasets
    !pip install --target=$pg_path transformers  
    !pip install --target=$pg_pat jiwer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Name: unicode
Version: 2.9
Summary: Display unicode character properties
Home-page: http://kassiopeia.juls.savba.sk/~garabik/software/unicode.html
Author: Radovan Garabik
Author-email: radovan.garabik@kassiopeia.juls.savba.sk
License: GNU GPL v3
Location: /usr/local/lib/python3.7/dist-packages
Requires: 
Required-by: 


In [ ]:
import itertools

In [ ]:
# unicode가 실행이 안되서 코드 전체를 여기다가 넣어 놈.
# from unicode import split_syllables, join_jamos
__all__ = ["split_syllable_char", "split_syllables",
           "join_jamos", "join_jamos_char",
           "CHAR_INITIALS", "CHAR_MEDIALS", "CHAR_FINALS"]

import itertools

INITIAL = 0x001
MEDIAL = 0x010
FINAL = 0x100
CHAR_LISTS = {
    INITIAL: list(map(chr, [
        0x3131, 0x3132, 0x3134, 0x3137, 0x3138, 0x3139,
        0x3141, 0x3142, 0x3143, 0x3145, 0x3146, 0x3147,
        0x3148, 0x3149, 0x314a, 0x314b, 0x314c, 0x314d,
        0x314e
    ])),
    MEDIAL: list(map(chr, [
        0x314f, 0x3150, 0x3151, 0x3152, 0x3153, 0x3154,
        0x3155, 0x3156, 0x3157, 0x3158, 0x3159, 0x315a,
        0x315b, 0x315c, 0x315d, 0x315e, 0x315f, 0x3160,
        0x3161, 0x3162, 0x3163
    ])),
    FINAL: list(map(chr, [
        0x3131, 0x3132, 0x3133, 0x3134, 0x3135, 0x3136,
        0x3137, 0x3139, 0x313a, 0x313b, 0x313c, 0x313d,
        0x313e, 0x313f, 0x3140, 0x3141, 0x3142, 0x3144,
        0x3145, 0x3146, 0x3147, 0x3148, 0x314a, 0x314b,
        0x314c, 0x314d, 0x314e
    ]))
}
CHAR_INITIALS = CHAR_LISTS[INITIAL]
CHAR_MEDIALS = CHAR_LISTS[MEDIAL]
CHAR_FINALS = CHAR_LISTS[FINAL]
CHAR_SETS = {k: set(v) for k, v in CHAR_LISTS.items()}
CHARSET = set(itertools.chain(*CHAR_SETS.values()))
CHAR_INDICES = {k: {c: i for i, c in enumerate(v)}
                for k, v in CHAR_LISTS.items()}


def is_hangul_syllable(c):
    return 0xac00 <= ord(c) <= 0xd7a3  # Hangul Syllables


def is_hangul_jamo(c):
    return 0x1100 <= ord(c) <= 0x11ff  # Hangul Jamo


def is_hangul_compat_jamo(c):
    return 0x3130 <= ord(c) <= 0x318f  # Hangul Compatibility Jamo


def is_hangul_jamo_exta(c):
    return 0xa960 <= ord(c) <= 0xa97f  # Hangul Jamo Extended-A


def is_hangul_jamo_extb(c):
    return 0xd7b0 <= ord(c) <= 0xd7ff  # Hangul Jamo Extended-B


def is_hangul(c):
    return (is_hangul_syllable(c) or
            is_hangul_jamo(c) or
            is_hangul_compat_jamo(c) or
            is_hangul_jamo_exta(c) or
            is_hangul_jamo_extb(c))


def is_supported_hangul(c):
    return is_hangul_syllable(c) or is_hangul_compat_jamo(c)


def check_hangul(c, jamo_only=False):
    if not ((jamo_only or is_hangul_compat_jamo(c)) or is_supported_hangul(c)):
        raise ValueError(f"'{c}' is not a supported hangul character. "
                         f"'Hangul Syllables' (0xac00 ~ 0xd7a3) and "
                         f"'Hangul Compatibility Jamos' (0x3130 ~ 0x318f) are "
                         f"supported at the moment.")


def get_jamo_type(c):
    check_hangul(c)
    assert is_hangul_compat_jamo(c), f"not a jamo: {ord(c):x}"
    return sum(t for t, s in CHAR_SETS.items() if c in s)


def split_syllable_char(c):
    """
    Splits a given korean syllable into its components. Each component is
    represented by Unicode in 'Hangul Compatibility Jamo' range.

    Arguments:
        c: A Korean character.

    Returns:
        A triple (initial, medial, final) of Hangul Compatibility Jamos.
        If no jamo corresponds to a position, `None` is returned there.

    Example:
        >>> split_syllable_char("안")
        ("ㅇ", "ㅏ", "ㄴ")
        >>> split_syllable_char("고")
        ("ㄱ", "ㅗ", None)
        >>> split_syllable_char("ㅗ")
        (None, "ㅗ", None)
        >>> split_syllable_char("ㅇ")
        ("ㅇ", None, None)
    """
    check_hangul(c)
    if len(c) != 1:
        raise ValueError("Input string must have exactly one character.")

    init, med, final = None, None, None
    if is_hangul_syllable(c):
        offset = ord(c) - 0xac00
        x = (offset - offset % 28) // 28
        init, med, final = x // 21, x % 21, offset % 28
        if not final:
            final = None
        else:
            final -= 1
    else:
        pos = get_jamo_type(c)
        if pos & INITIAL == INITIAL:
            pos = INITIAL
        elif pos & MEDIAL == MEDIAL:
            pos = MEDIAL
        elif pos & FINAL == FINAL:
            pos = FINAL
        idx = CHAR_INDICES[pos][c]
        if pos == INITIAL:
            init = idx
        elif pos == MEDIAL:
            med = idx
        else:
            final = idx
    return tuple(CHAR_LISTS[pos][idx] if idx is not None else None
                 for pos, idx in
                 zip([INITIAL, MEDIAL, FINAL], [init, med, final]))


def split_syllables(s, ignore_err=True, pad=None):
    """
    Performs syllable-split on a string.

    Arguments:
        s (str): A string (possibly mixed with non-Hangul characters).
        ignore_err (bool): If set False, it ensures that all characters in
            the string are Hangul-splittable and throws a ValueError otherwise.
            (default: True)
        pad (str): Pad empty jamo positions (initial, medial, or final) with
            `pad` character. This is useful for cases where fixed-length
            strings are needed. (default: None)

    Returns:
        Hangul-split string

    Example:
        >>> split_syllables("안녕하세요")
        "ㅇㅏㄴㄴㅕㅇㅎㅏㅅㅔㅇㅛ"
        >>> split_syllables("안녕하세요~~", ignore_err=False)
        ValueError: encountered an unsupported character: ~ (0x7e)
        >>> split_syllables("안녕하세요ㅛ", pad="x")
        'ㅇㅏㄴㄴㅕㅇㅎㅏxㅅㅔxㅇㅛxxㅛx'
    """

    def try_split(c):
        try:
            return split_syllable_char(c)
        except ValueError:
            if ignore_err:
                return (c,)
            raise ValueError(f"encountered an unsupported character: "
                             f"{c} (0x{ord(c):x})")

    s = map(try_split, s)
    if pad is not None:
        tuples = map(lambda x: tuple(pad if y is None else y for y in x), s)
    else:
        tuples = map(lambda x: filter(None, x), s)
    return "".join(itertools.chain(*tuples))


def join_jamos_char(init, med, final=None):
    """
    Combines jamos into a single syllable.

    Arguments:
        init (str): Initial jao.
        med (str): Medial jamo.
        final (str): Final jamo. If not supplied, the final syllable is made
            without the final. (default: None)

    Returns:
        A Korean syllable.
    """
    chars = (init, med, final)
    for c in filter(None, chars):
        check_hangul(c, jamo_only=True)

    idx = tuple(CHAR_INDICES[pos][c] if c is not None else c
                for pos, c in zip((INITIAL, MEDIAL, FINAL), chars))
    init_idx, med_idx, final_idx = idx
    # final index must be shifted once as
    # final index with 0 points to syllables without final
    final_idx = 0 if final_idx is None else final_idx + 1
    return chr(0xac00 + 28 * 21 * init_idx + 28 * med_idx + final_idx)


def join_jamos(s, ignore_err=True):
    """
    Combines a sequence of jamos to produce a sequence of syllables.

    Arguments:
        s (str): A string (possible mixed with non-jamo characters).
        ignore_err (bool): If set False, it will ensure that all characters
            will be consumed for the making of syllables. It will throw a
            ValueError when it fails to do so. (default: True)

    Returns:
        A string

    Example:
        >>> join_jamos("ㅇㅏㄴㄴㅕㅇㅎㅏㅅㅔㅇㅛ")
        "안녕하세요"
        >>> join_jamos("ㅇㅏㄴㄴㄴㅕㅇㅎㅏㅅㅔㅇㅛ")
        "안ㄴ녕하세요"
        >>> join_jamos()
    """
    last_t = 0
    queue = []
    new_string = ""

    def flush(n=0):
        new_queue = []
        while len(queue) > n:
            new_queue.append(queue.pop())
        if len(new_queue) == 1:
            if not ignore_err:
                raise ValueError(f"invalid jamo character: {new_queue[0]}")
            result = new_queue[0]
        elif len(new_queue) >= 2:
            try:
                result = join_jamos_char(*new_queue)
            except (ValueError, KeyError):
                # Invalid jamo combination
                if not ignore_err:
                    raise ValueError(f"invalid jamo characters: {new_queue}")
                result = "".join(new_queue)
        else:
            result = None
        return result

    for c in s:
        if c not in CHARSET:
            if queue:
                new_c = flush() + c
            else:
                new_c = c
            last_t = 0
        else:
            t = get_jamo_type(c)
            new_c = None
            if t & FINAL == FINAL:
                if not (last_t == MEDIAL):
                    new_c = flush()
            elif t == INITIAL:
                new_c = flush()
            elif t == MEDIAL:
                if last_t & INITIAL == INITIAL:
                    new_c = flush(1)
                else:
                    new_c = flush()
            last_t = t
            queue.insert(0, c)
        if new_c:
            new_string += new_c
    if queue:
        new_string += flush()
    return new_string

## Create Wav2Vec2CTCTokenizer

In [ ]:
from datasets import load_dataset
import pandas as pd
import numpy as np
import re
import json

from tqdm import tqdm

In [ ]:
dir_path = ['./dataset/KlecSpeech/csv/', './dataset/csv/']
file_prefix = ['klecspeech_ko_500', 'order_speech_ko1000']

In [ ]:
# 모든 csv 파일 리스트를 가져온다.
def get_all_filelist(dir_path):
    csv_files = []
    for i in range(len(dir_path)):
        csv_list = os.listdir(dir_path[i])
        for file in csv_list:
            csv_files.append(dir_path[i]+file)
    return csv_files

In [ ]:
# 결과를 확인한다.
all_files = get_all_filelist(dir_path)
all_files

['./dataset/KlecSpeech/csv/klecspeech_ko_500_000.csv',
 './dataset/KlecSpeech/csv/klecspeech_ko_500_001.csv',
 './dataset/KlecSpeech/csv/klecspeech_ko_500_002.csv',
 './dataset/KlecSpeech/csv/klecspeech_ko_500_003.csv',
 './dataset/KlecSpeech/csv/klecspeech_ko_500_004.csv',
 './dataset/KlecSpeech/csv/klecspeech_ko_500_005.csv',
 './dataset/KlecSpeech/csv/klecspeech_ko_500_006.csv',
 './dataset/KlecSpeech/csv/klecspeech_ko_500_007.csv',
 './dataset/KlecSpeech/csv/klecspeech_ko_500_008.csv',
 './dataset/KlecSpeech/csv/klecspeech_ko_500_009.csv',
 './dataset/KlecSpeech/csv/klecspeech_ko_500_010.csv',
 './dataset/KlecSpeech/csv/klecspeech_ko_500_011.csv',
 './dataset/KlecSpeech/csv/klecspeech_ko_500_012.csv',
 './dataset/KlecSpeech/csv/klecspeech_ko_500_013.csv',
 './dataset/KlecSpeech/csv/klecspeech_ko_500_014.csv',
 './dataset/KlecSpeech/csv/klecspeech_ko_500_015.csv',
 './dataset/KlecSpeech/csv/klecspeech_ko_500_016.csv',
 './dataset/KlecSpeech/csv/klecspeech_ko_500_017.csv',
 './datase

In [ ]:
# get_filelist 
# start_idx부터 step_num 건수 만큼 파일리스트 추가
def get_filelist(csv_list, start_idx, step_num):

    end_idx = len(csv_list)
    new_list = []
    if len(csv_list) == 0:
        print("array size is 0")
        return None
    else:
        if len(csv_list) < start_idx + step_num:
            end_idx = len(csv_list)
        else:
            end_idx = start_idx + step_num

        for i in range(start_idx, end_idx):
            print("file appned : ", i, " ",csv_list[i])
            new_list.append(csv_list[i])
        return new_list

In [ ]:
def load_datasets(files):
    # files가 list로 되어 있으면 한번에 합쳐서 load해온다.
    dataset = load_dataset('csv',data_files=files, split='train', sep='\t')
    return dataset

In [ ]:
text = "text"

In [ ]:
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�]'

def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower() + " "
    return batch

In [ ]:
import json
import time

# 한 번 실행에 몇 개의 파일을 묶을 것인가?
num_of_files = 2

start = time.time()  # 시작 시간 저장
# 작업 코드
for n_cnt in range(0, len(all_files), num_of_files):
    filelist = get_filelist(all_files, n_cnt, num_of_files)
    print(filelist)
    data_csv = load_datasets(filelist)
    remove_spectial_char_data = data_csv.map(remove_special_characters )


    target_data = remove_spectial_char_data
    jamos = []
    for i in tqdm(range(len(target_data))):
        split_text = split_syllables(target_data[i][text])
        all_text = " ".join(split_text)
        jamos.extend(list(set(all_text)))

    vocab_list = list(set(jamos))

    # 0~4까지는 토큰 예약으로 5번 부터 시작한다.
    vocab_dict = {v: k+5 for k, v in enumerate(vocab_list)}

    vocab_dict["<pad>"] = 0
    vocab_dict["<s>"] = 1
    vocab_dict["</s>"] = 2
    vocab_dict["<unk>"] = 3
    vocab_dict["|"] = 4
    vocab_dict["<b>"] = 5
    vocab_dict["<n>"] = 6
    vocab_dict["<l>"] = 7
    len(vocab_dict)

    print(vocab_dict)
    with open(f'vocab_{n_cnt:03d}.json_temp', 'w') as vocab_file:
        json.dump(vocab_dict, vocab_file)
        print(f"{vocab_file.filename} is created")


print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

file appned :  0   ./dataset/KlecSpeech/csv/klecspeech_ko_500_000.csv
file appned :  1   ./dataset/KlecSpeech/csv/klecspeech_ko_500_001.csv
['./dataset/KlecSpeech/csv/klecspeech_ko_500_000.csv', './dataset/KlecSpeech/csv/klecspeech_ko_500_001.csv']


  0%|          | 0/1000 [00:00<?, ?ex/s]

100%|██████████| 1000/1000 [00:01<00:00, 745.91it/s]


{'ㅉ': 5, 'ㅏ': 6, 'ㅁ': 7, 'ㄱ': 8, 'ㄺ': 9, 'ㅛ': 10, 'ㄸ': 11, 'ㅑ': 12, 'ㅀ': 13, 'ㅅ': 14, 'ㅕ': 15, 'ㅌ': 16, 'ㅐ': 17, 'ㅘ': 18, 'ㄵ': 19, 'ㄼ': 20, ' ': 21, 'ㅞ': 22, 'ㅔ': 23, 'ㅋ': 24, 'ㄷ': 25, 'ㅜ': 26, 'ㅈ': 27, 'ㅟ': 28, 'ㅚ': 29, 'ㄲ': 30, 'ㅃ': 31, 'ㅊ': 32, 'ㄶ': 33, 'ㅎ': 34, 'ㅂ': 35, 'ㄹ': 36, 'ㅍ': 37, 'ㅖ': 38, 'ㅢ': 39, 'ㅓ': 40, 'ㅡ': 41, 'ㅙ': 42, 'ㅣ': 43, 'ㅗ': 44, 'ㅝ': 45, 'ㅇ': 46, 'ㅄ': 47, 'ㄴ': 48, 'ㅒ': 49, 'ㅠ': 50, 'ㅆ': 51, '|': 4, '<pad>': 0, '<s>': 1, '</s>': 2, '<unk>': 3}
<_io.TextIOWrapper name='vocab_000.json_temp' mode='w' encoding='UTF-8'> is created
file appned :  2   ./dataset/KlecSpeech/csv/klecspeech_ko_500_002.csv
file appned :  3   ./dataset/KlecSpeech/csv/klecspeech_ko_500_003.csv
['./dataset/KlecSpeech/csv/klecspeech_ko_500_002.csv', './dataset/KlecSpeech/csv/klecspeech_ko_500_003.csv']


  0%|          | 0/1000 [00:00<?, ?ex/s]

...

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-1f516d0570fadd14/0.0.0/51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58. Subsequent calls will reuse this data.


  0%|          | 0/1000 [00:00<?, ?ex/s]

100%|██████████| 1000/1000 [00:01<00:00, 573.19it/s]


{'ㅉ': 5, 'ㅏ': 6, 'ㅁ': 7, 'ㄱ': 8, 'ㄺ': 9, 'ㅛ': 10, 'ㄸ': 11, 'ㅑ': 12, 'ㅀ': 13, 'ㄿ': 14, 'ㅅ': 15, 'ㅕ': 16, 'ㅌ': 17, 'ㅐ': 18, 'ㅘ': 19, 'ㄵ': 20, 'ㅞ': 21, ' ': 22, 'ㄼ': 23, 'ㅔ': 24, 'ㅋ': 25, 'ㄷ': 26, 'ㅜ': 27, 'ㅈ': 28, 'ㅟ': 29, 'ㅚ': 30, 'ㄲ': 31, 'ㅃ': 32, 'ㄻ': 33, 'ㅊ': 34, 'ㄶ': 35, 'ㅎ': 36, 'ㅂ': 37, 'ㄹ': 38, 'ㅍ': 39, 'ㅖ': 40, 'ㅢ': 41, 'ㅡ': 42, 'ㅓ': 43, 'ㅙ': 44, 'ㅗ': 45, 'ㅣ': 46, 'ㅝ': 47, 'ㅇ': 48, 'ㅄ': 49, 'ㄴ': 50, 'ㅒ': 51, 'ㅠ': 52, 'ㅆ': 53, '|': 4, '<pad>': 0, '<s>': 1, '</s>': 2, '<unk>': 3}
<_io.TextIOWrapper name='vocab_038.json_temp' mode='w' encoding='UTF-8'> is created
file appned :  40   ./dataset/KlecSpeech/csv/klecspeech_ko_500_040.csv
file appned :  41   ./dataset/KlecSpeech/csv/klecspeech_ko_500_041.csv
['./dataset/KlecSpeech/csv/klecspeech_ko_500_040.csv', './dataset/KlecSpeech/csv/klecspeech_ko_500_041.csv']


OSError: ignored

In [ ]:
import glob
json_files = glob.glob('*.json_temp')
print(json_files)

vocab_all = {}
for filename in json_files:
    with open(filename, 'r') as vocab_file:
        vocab_dict = json.load(vocab_file)
        print(type(vocab_dict))
        vocab_all.update(vocab_dict)

# 번호 재배열
vocab_all = {v: k for k, v in enumerate(vocab_all)}
print(len(vocab_all))

# print(vocab_all)

with open(f'vocab.json', 'w') as vocab_file:  
    json.dump(vocab_all, vocab_file)


['vocab_016.json_temp', 'vocab_014.json_temp', 'vocab_018.json_temp', 'vocab_022.json_temp', 'vocab_028.json_temp', 'vocab_034.json_temp', 'vocab_006.json_temp', 'vocab_020.json_temp', 'vocab_030.json_temp', 'vocab_038.json_temp', 'vocab_008.json_temp', 'vocab_000.json_temp', 'vocab_024.json_temp', 'vocab_032.json_temp', 'vocab_012.json_temp', 'vocab_036.json_temp', 'vocab_026.json_temp', 'vocab_004.json_temp', 'vocab_010.json_temp', 'vocab_002.json_temp']
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
56
